### Import Section

In [ ]:
# import matplotlib.pyplot as plt
# import math


# import numpy as np
# import os
# import pandas as pd


# from tensorflow.keras import Model, Sequential

# from tensorflow.keras.applications.resnet50 import ResNet50



In [ ]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


Compute dtype: float16
Variable dtype: float32


### Mounting Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Assigning Directories

In [ ]:
directory      = "/content/drive/MyDrive/Extra/train.csv"

### We have two tasks.
Task-1: Recognize each bangla character. Multi-class classification.

Task-2: Recognize male or female handwritting. Binary Classification.

Example Code - https://towardsdatascience.com/multi-task-learning-for-computer-vision-classification-with-keras-36c52e6243d2

In [ ]:
image_size = (28,28)
batch_size = 8

### Dataset Description

All Images are 28x28 pixels.

Dataset devided into train, test, validation

In [ ]:
# Library
import pandas as pd
import numpy as np

# Load Data
train = pd.read_csv("train.csv")

# Separating Data and Label
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1)

# Normalize the data
X_train = X_train / 255.0

# Reshape the array into 28 x 28 pixel
X_train = X_train.values.reshape(-1,28,28,1)

In [ ]:
# # experimental
import tensorflow as tf


train_generator = tf.keras.utils.image_dataset_from_directory(train_directory,
                                                              labels='inferred',
                                                              color_mode='grayscale',
                                                            shuffle=True,
                                                            batch_size=batch_size,
                                                            image_size=image_size)

test_generator = tf.keras.utils.image_dataset_from_directory(test_directory,
                                                             labels='inferred',
                                                             color_mode='grayscale',
                                                            shuffle=True,
                                                            batch_size=batch_size,
                                                            image_size=image_size)
                                                            
validation_generator = tf.keras.utils.image_dataset_from_directory(validation_directory,
                                                                   labels='inferred',
                                                                   color_mode='grayscale',
                                                                 shuffle=True,
                                                                 batch_size=batch_size,
                                                                 image_size=image_size)
                                                            
class_names = train_generator.class_names

print('Number of training batches: %d' % tf.data.experimental.cardinality(train_generator).numpy())
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_generator).numpy())
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_generator).numpy())


Found 2000 files belonging to 2 classes.
Found 300 files belonging to 2 classes.
Found 600 files belonging to 2 classes.
Number of training batches: 250
Number of test batches: 38
Number of validation batches: 75


In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator


datagen_train = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = False 
)

datagen_test = ImageDataGenerator(rescale = 1./255)

datagen_validation = ImageDataGenerator(rescale = 1./255)

train_generator = datagen_train.flow_from_directory(
    directory = train_directory,
    target_size = image_size,
    batch_size = batch_size,
    shuffle = True,
    color_mode='grayscale',
    follow_links=True
    )

test_generator = datagen_test.flow_from_directory(
    directory = test_directory,
    target_size = image_size,
    batch_size = batch_size,
    shuffle = True,
    color_mode='grayscale',
    follow_links=True
    )

validation_generator = datagen_validation.flow_from_directory(
    directory = validation_directory,
    target_size = image_size,
    batch_size = batch_size,
    shuffle = True,
    color_mode='grayscale',
    follow_links=True
    )

Found 2000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from keras import layers
from keras.layers import Dense, Flatten, Dropout, Conv2D , MaxPooling2D

inputs = tf.keras.layers.Input(shape=(28, 28, 1), name='input')

main_branch = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1)(inputs)
main_branch = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(main_branch)
main_branch = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1)(main_branch)
main_branch = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(main_branch)
main_branch = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1)(main_branch)
main_branch = tf.keras.layers.Flatten()(main_branch)
main_branch = tf.keras.layers.Dense(512, activation='relu')(main_branch)


task_1_branch = tf.keras.layers.Dense(512, activation='relu')(main_branch)
task_1_branch = tf.keras.layers.Dense(256, activation='relu')(main_branch)
task_1_branch = tf.keras.layers.Dense(128, activation='relu')(task_1_branch)
task_1_branch = tf.keras.layers.Dense(10, activation='softmax', name='task_1_output')(task_1_branch)


task_2_branch = tf.keras.layers.Dense(512, activation='relu')(main_branch)
task_2_branch = tf.keras.layers.Dense(256, activation='relu')(task_2_branch)
task_2_branch = tf.keras.layers.Dense(100, activation='relu')(task_2_branch)
task_2_branch = tf.keras.layers.Dense(2, activation='sigmoid', name='task_2_output')(task_2_branch)

model = tf.keras.Model(inputs = inputs, outputs = [task_1_branch, task_2_branch])

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=1e-5)
# loss = 'categorical_crossentropy'

loss={'task_1_output': 'categorical_crossentropy',
      'task_2_output': 'binary_crossentropy'}

# metrics = ['categorical_accuracy']
#metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 26, 26, 32)   320         ['input[0][0]']                  
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 13, 13, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 11, 11, 64)   18496       ['max_pooling2d[0][0]']          
                                                                                              

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!curl https://raw.githubusercontent.com/dexterfichuk/GoogleDriveCheckpoint/master/google_drive_checkpoint.py -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4522  100  4522    0     0  30972      0 --:--:-- --:--:-- --:--:-- 30972


In [ ]:
pip install httplib2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install google-api-python-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google_drive_checkpoint import GoogleDriveCheckpoint

checkpoint = GoogleDriveCheckpoint('testModel.h5', drive, save_best_only=True, 
                                   save_weights_only=False, 
                                   monitor = 'val_accuracy', mode = 'auto',
                                   verbose=1)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop = EarlyStopping(
    monitor = 'val_accuracy', 
    min_delta = 0, #val_acc
    patience = 10, 
    verbose = 1, 
    mode = 'auto'
)

In [ ]:
epochs = 20
# steps_per_epoch = (train_generator.n / batch_size)
# steps_test = test_generator.n / batch_size
# steps_validation  = validation_generator.n / batch_size





history = model.fit_generator(
    generator=train_generator,
    epochs = epochs, 
    # steps_per_epoch = steps_per_epoch,
    validation_data = validation_generator,
    # validation_steps = steps_test,
    callbacks = [checkpoint, earlystop]
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  app.launch_new_instance()


ValueError: ignored

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train Accuracy","Validation Accuracy"], loc = 'lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train Loss","Validation Loss"], loc = 'upper right')
plt.show()

In [ ]:
finaltestCNV_dir = test_directory
datagen_finaltestCNV_dir = ImageDataGenerator(rescale=1./255)

generator_finaltestCNV_dir = datagen_finaltestCNV_dir.flow_from_directory(directory=finaltestCNV_dir,
                                                  target_size=image_size,
                                                  batch_size=batch_size,
                                                  shuffle=False)
steps_finaltestCNV_dir = generator_finaltestCNV_dir.n / batch_size

y_pred = model.predict_generator(generator_finaltestCNV_dir,steps = steps_finaltestCNV_dir)
y_pred

In [ ]:
predicted_class_indices=np.argmax(y_pred,axis=1)
print(predicted_class_indices)

In [ ]:
test_label = []
ADILabel = 0
BACKLabel = 1
DEBLabel = 2
LYMLabel = 3
MUCLabel = 4
MUSLabel = 5
NORMLabel = 6
STRLabel = 7
TUMLabel = 8



for i in range(1000):
    test_label.append(ADILabel)

for i in range(1069):
    test_label.append(BACKLabel)

for i in range(1207):
    test_label.append(DEBLabel)

for i in range(1233):
    test_label.append(LYMLabel)

for i in range(1026):
    test_label.append(MUCLabel)


for i in range(1364):
    test_label.append(MUSLabel)


for i in range(877):
    test_label.append(NORMLabel)

for i in range(1164):
    test_label.append(STRLabel)


for i in range(1452):
    test_label.append(TUMLabel)


print(test_label)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import itertools

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
df_cm = pd.DataFrame(array, index = ["ADI", "BACK","DEB","LYM","MUC","MUS","NORM","STR","TUM"],
                  columns = ["ADI", "BACK","DEB","LYM","MUC","MUS","NORM","STR","TUM"])
plt.figure(figsize = (10,10))
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')